## 1. Import Required Libraries and Setup

In [ ]:
# Add current directory to path
import sys
import os
sys.path.insert(0, '/Users/eemanadnan/Documents/MS-AI/Assignments/structure-from-motion-master')

# Core libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle
import logging
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

print("✓ Libraries imported successfully")

## 2. Load and Prepare Your Images

In [ ]:
# Setup paths
SOURCE_IMAGE_DIR = '/Users/eemanadnan/Documents/MS-AI/Assignments/3D_Scene_Reconstruction/images_folder'
SFM_ROOT = '/Users/eemanadnan/Documents/MS-AI/Assignments/structure-from-motion-master'
IMAGES_DIR = os.path.join(SFM_ROOT, 'images')

# Create images directory if it doesn't exist
os.makedirs(IMAGES_DIR, exist_ok=True)

print(f"Source images: {SOURCE_IMAGE_DIR}")
print(f"SfM root: {SFM_ROOT}")
print(f"Images will be in: {IMAGES_DIR}")

In [ ]:
# Find and list JPEG images
image_files = sorted([f for f in os.listdir(SOURCE_IMAGE_DIR) 
                      if f.lower().endswith(('.jpg', '.jpeg'))])

print(f"Found {len(image_files)} JPEG images")
print(f"\nFirst 10 images:")
for i, f in enumerate(image_files[:10]):
    print(f"  {i+1}. {f}")
print(f"  ... and {len(image_files)-10} more" if len(image_files) > 10 else "")

In [ ]:
# Copy images to SfM directory
# You can change NUM_IMAGES to test with fewer images first
NUM_IMAGES = 20  # Start with 20 for testing, increase for full reconstruction

import shutil

selected_images = image_files[:NUM_IMAGES]
print(f"Copying {len(selected_images)} images to {IMAGES_DIR}...")

for idx, img_file in enumerate(selected_images):
    src = os.path.join(SOURCE_IMAGE_DIR, img_file)
    dst = os.path.join(IMAGES_DIR, img_file)
    if not os.path.exists(dst):
        shutil.copy2(src, dst)
    if (idx + 1) % 5 == 0:
        print(f"  Copied {idx + 1}/{len(selected_images)}")

print(f"✓ Done! Images ready in {IMAGES_DIR}")

In [ ]:
# Display sample images
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Sample Images from Your Sequence', fontsize=16)

sample_indices = [0, len(selected_images)//5, 2*len(selected_images)//5, 
                  3*len(selected_images)//5, 4*len(selected_images)//5, 
                  len(selected_images)-1]

for idx, ax in enumerate(axes.flat):
    img_idx = sample_indices[idx]
    img_path = os.path.join(IMAGES_DIR, selected_images[img_idx])
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax.imshow(img_rgb)
    ax.set_title(f"Image {img_idx+1}/{len(selected_images)}")
    ax.axis('off')

plt.tight_layout()
plt.show()

print(f"✓ Displayed {len(sample_indices)} sample images")

## 3. Create Camera Calibration

In [ ]:
# Create camera intrinsic matrix K
# These are reasonable defaults for iPhone images (3024x4032)
focal_length = 1000.0  # pixels
cx = 3024 / 2  # principal point X
cy = 4032 / 2  # principal point Y

K = np.array([
    [focal_length, 0, cx],
    [0, focal_length, cy],
    [0, 0, 1]
], dtype=float)

# Save K.txt
K_path = os.path.join(IMAGES_DIR, 'K.txt')
np.savetxt(K_path, K)

print("Camera Intrinsic Matrix K:")
print(K)
print(f"\n✓ Saved to {K_path}")
print(f"\nCamera parameters:")
print(f"  Focal length: {focal_length} pixels")
print(f"  Principal point: ({cx}, {cy})")

## 4. Run SfM Reconstruction

This will execute the complete Structure from Motion pipeline:
- Feature extraction (SIFT)
- Feature matching between images
- Baseline selection
- Camera pose estimation
- 3D point triangulation
- Bundle adjustment optimization

In [ ]:
# Import SfM modules
try:
    from view import *
    from match import *
    from sfm import SFM
    print("✓ SfM modules imported successfully")
except ImportError as e:
    print(f"Error importing SfM modules: {e}")
    print("Make sure you're running this notebook from the SfM directory")

In [ ]:
# Create views (load images and extract features)
print(f"Creating views from {NUM_IMAGES} images...")
print("This may take a few minutes for feature extraction...\n")

views = create_views(SFM_ROOT, 'jpg')
print(f"\n✓ Created {len(views)} views")
print(f"\nView information:")
for i, view in enumerate(views[:3]):
    print(f"  View {i}: {view.name}, Image size: {view.image.shape}, Features: {len(view.keypoints)}")
if len(views) > 3:
    print(f"  ... and {len(views)-3} more views")

In [ ]:
# Create feature matches
print("Matching features between images...")
print("This may take a while...\n")

matches = create_matches(views)
print(f"\n✓ Feature matching complete")
print(f"Total matches created: {len(matches)}")

In [ ]:
# Load camera matrix
K = np.loadtxt(os.path.join(SFM_ROOT, 'images', 'K.txt'))
print("Camera matrix K loaded:")
print(K)

In [ ]:
# Run SfM reconstruction
print("Running SfM reconstruction...")
print("This involves:")
print("  - Baseline selection")
print("  - Camera pose estimation")
print("  - 3D point triangulation")
print("  - Bundle adjustment\n")

sfm = SFM(views, matches, K)
sfm.reconstruct()

print(f"\n✓ Reconstruction complete!")
print(f"Total 3D points: {len(sfm.points_3D)}")
print(f"Output saved to: {sfm.results_path}")

## 5. Visualize Results

In [ ]:
# Check output files
output_dir = sfm.results_path
if os.path.exists(output_dir):
    output_files = os.listdir(output_dir)
    print(f"Output files in {output_dir}:")
    for f in sorted(output_files):
        file_path = os.path.join(output_dir, f)
        file_size = os.path.getsize(file_path)
        print(f"  - {f} ({file_size/1024:.1f} KB)")
else:
    print(f"Output directory not found: {output_dir}")

In [ ]:
# Plot 3D points (using matplotlib for basic visualization)
if len(sfm.points_3D) > 0:
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot points
    points = sfm.points_3D
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], 
              c='blue', marker='.', s=1, alpha=0.6)
    
    # Plot camera centers (estimated poses)
    for i, view in enumerate(views):
        if not np.all(view.R == 0):  # If pose has been computed
            # Camera center C = -R^T @ t
            C = -view.R.T @ view.t
            ax.scatter(C[0], C[1], C[2], c='red', s=100, marker='o')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'3D Reconstruction\nPoints: {len(sfm.points_3D)}')
    
    plt.tight_layout()
    plt.show()
    
    print(f"✓ Visualized {len(sfm.points_3D)} 3D points")
else:
    print("No 3D points reconstructed")

In [ ]:
# Summary statistics
print("\n=== SfM Reconstruction Summary ===")
print(f"Input images: {NUM_IMAGES}")
print(f"3D points reconstructed: {len(sfm.points_3D)}")
print(f"Camera poses estimated: {sum(1 for v in views if not np.all(v.R == 0))}")
if sfm.errors:
    print(f"Mean reprojection error: {np.mean(sfm.errors):.4f} pixels")
print(f"\nOutput directory: {output_dir}")
print(f"\nNext steps:")
print(f"1. View the .ply files in MeshLab or CloudCompare")
print(f"2. Adjust camera calibration if needed")
print(f"3. Run with more images for better coverage")

## 6. Next Steps

### To view your 3D reconstruction:

1. **Using MeshLab** (Recommended):
   - Download from https://www.meshlab.net/
   - Open any `.ply` file in the `points/` directory

2. **Using CloudCompare**:
   - Download from https://www.cloudcompare.org/
   - Open the point cloud files

3. **Using Python** (Open3D):
   ```python
   import open3d as o3d
   pcd = o3d.io.read_point_cloud('points/final_cloud.ply')
   o3d.visualization.draw_geometries([pcd])
   ```

### To improve reconstruction:

- Increase `NUM_IMAGES` to use more images
- Verify images have good overlap
- Check camera calibration values
- Try different feature types (SURF, ORB)

### To run from command line:

```bash
python run_sfm.py --num_images 50 --feat_type sift
```

See `QUICK_START.md` and `SETUP_GUIDE.md` for more details!